In [1]:
dataset = spark.read.csv("file:///home/guri/Desktop/Wine-Dataset-Classification-Using-Spark-Mlib/wine.csv",header=True,inferSchema=True)

In [2]:
from pyspark.ml.classification import LogisticRegression

In [3]:
dataset.printSchema()

root
 |-- Class: integer (nullable = true)
 |-- Alcohol: double (nullable = true)
 |-- Malic acid: double (nullable = true)
 |-- Ash: double (nullable = true)
 |-- Alcalinity of ash: double (nullable = true)
 |-- Magnesium: integer (nullable = true)
 |-- Total phenols: double (nullable = true)
 |-- Flavanoids: double (nullable = true)
 |-- Nonflavanoid phenols: double (nullable = true)
 |-- Proanthocyanins: double (nullable = true)
 |-- Color intensity: double (nullable = true)
 |-- Hue: double (nullable = true)
 |-- OD280/OD315 of diluted wines: double (nullable = true)
 |-- Proline: integer (nullable = true)



In [4]:
dataset.show(5)

+-----+-------+----------+----+-----------------+---------+-------------+----------+--------------------+---------------+---------------+----+----------------------------+-------+
|Class|Alcohol|Malic acid| Ash|Alcalinity of ash|Magnesium|Total phenols|Flavanoids|Nonflavanoid phenols|Proanthocyanins|Color intensity| Hue|OD280/OD315 of diluted wines|Proline|
+-----+-------+----------+----+-----------------+---------+-------------+----------+--------------------+---------------+---------------+----+----------------------------+-------+
|    1|  14.23|      1.71|2.43|             15.6|      127|          2.8|      3.06|                0.28|           2.29|           5.64|1.04|                        3.92|   1065|
|    1|   13.2|      1.78|2.14|             11.2|      100|         2.65|      2.76|                0.26|           1.28|           4.38|1.05|                         3.4|   1050|
|    1|  13.16|      2.36|2.67|             18.6|      101|          2.8|      3.24|                

In [5]:
from pyspark.sql.functions import isnan, when, count, col

dataset.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in dataset.columns]).show()

+-----+-------+----------+---+-----------------+---------+-------------+----------+--------------------+---------------+---------------+---+----------------------------+-------+
|Class|Alcohol|Malic acid|Ash|Alcalinity of ash|Magnesium|Total phenols|Flavanoids|Nonflavanoid phenols|Proanthocyanins|Color intensity|Hue|OD280/OD315 of diluted wines|Proline|
+-----+-------+----------+---+-----------------+---------+-------------+----------+--------------------+---------------+---------------+---+----------------------------+-------+
|    0|      0|         0|  0|                0|        0|            0|         0|                   0|              0|              0|  0|                           0|      0|
+-----+-------+----------+---+-----------------+---------+-------------+----------+--------------------+---------------+---------------+---+----------------------------+-------+



In [6]:
dataset.columns

['Class',
 'Alcohol',
 'Malic acid',
 'Ash',
 'Alcalinity of ash',
 'Magnesium',
 'Total phenols',
 'Flavanoids',
 'Nonflavanoid phenols',
 'Proanthocyanins',
 'Color intensity',
 'Hue',
 'OD280/OD315 of diluted wines',
 'Proline']

In [7]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
 
assembler=VectorAssembler(inputCols=['Alcohol',
 'Malic acid',
 'Ash',
 'Alcalinity of ash',
 'Magnesium',
 'Total phenols',
 'Flavanoids',
 'Nonflavanoid phenols',
 'Proanthocyanins',
 'Color intensity',
 'Hue',
 'OD280/OD315 of diluted wines',
 'Proline'],outputCol='features')
 
output_data=assembler.transform(dataset)

In [8]:
final_data=output_data.select('features','Class')    
train,test=final_data.randomSplit([0.7,0.3])
model=LogisticRegression(labelCol='Class')           
model=model.fit(train)        
summary=model.summary 
summary.predictions.describe().show()

+-------+------------------+------------------+
|summary|             Class|        prediction|
+-------+------------------+------------------+
|  count|               124|               124|
|   mean|1.9758064516129032|1.9758064516129032|
| stddev|0.7908389049715342|0.7908389049715342|
|    min|               1.0|               1.0|
|    max|               3.0|               3.0|
+-------+------------------+------------------+



In [31]:
from pyspark.mllib.evaluation import BinaryClassificationMetrics
predictionAndLabels = test.map(lambda lp: (float(model.predict(lp.features)), lp.label))
metrics = BinaryClassificationMetrics(predictionAndLabels)

AttributeError: 'DataFrame' object has no attribute 'map'